In [35]:
ls

T403_在庫推移のグラフ可視化.ipynb
欠品リスク.ipynb
設計値変更後_データ作成_滞留日数.ipynb
設計値変更前_データ作成_滞留日数.ipynb
設計値変更後_データ作成_滞留日数_各工程.ipynb
かんばん数計算.ipynb
設計値変更前と後の比較.ipynb


In [121]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [122]:
#ライブラリのimport
import pandas as pd
import numpy as np
%matplotlib inline
#from matplotlib import pyplot as pyp
import matplotlib.pyplot as plt

In [123]:
#データ準備
#手順
#0.所在管理LTと行数合わせる必要ある。作り方は所在管理LTのコピー
#1.発注日時がNULL消す
#2.シム消す
#3.回収日時が7月8月のものは消す
#4.〇~〇LTから～を消す
#5.別ファイルにコピー
pre_zaikodata = pd.read_csv('..//data//20231006_所在管理LT.csv',encoding='shift_jis')
#ひとまずダミーデータで。
#pre_zaikodata = pd.read_csv('..//data//pre_zaikodata_20230828.csv',encoding='shift_jis')
#pre_zaikodata = pd.read_csv('sample1.csv',encoding='shift_jis')
pre_zaikodata.head(300)

,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,...,回収日時,発注取消日時,発注〜印刷LT,発注〜検収LT,発注〜順立装置入庫LT,発注〜順立装置出庫LT,発注〜組立LT,発注〜回収LT,更新日時,長期滞留フラグ
0,1Z23J00061273,XCGG850,10429,35482TFA010,ｸﾗﾝﾌﾟｾﾝｻ-,1000,サトープレス工業（株）,本社工場,回収済,2023/9/26,...,2023/10/6 12:18,< NULL >,0.11,3.93,4.09,9.96,< NULL >,14.01,2023/10/6 12:18,< NULL >
1,1Z23J00090101,XCGP260,10429,35482TFA010,ｸﾗﾝﾌﾟｾﾝｻ-,1000,サトープレス工業（株）,本社工場,順立装置内,2023/9/29,...,< NULL >,< NULL >,0.13,2.26,9.03,< NULL >,< NULL >,< NULL >,2023/10/6 12:11,< NULL >
2,1Z23J00091003,XCGR590,10416,9030106A018,ﾘﾝｸﾞO,2000,ＮＯＫ（株）,< NULL >,順立装置内,2023/9/29,...,< NULL >,< NULL >,0.15,2.27,9.03,< NULL >,< NULL >,< NULL >,2023/10/6 12:11,< NULL >
3,1Z23J00090128,XCGN430,10149,9056450A003,ｼﾑ,100,（株）ムロコーポレーション,< NULL >,回収済,2023/9/27,...,2023/10/6 11:33,< NULL >,0.1,0.86,1.14,8.92,< NULL >,9.22,2023/10/6 11:33,< NULL >
4,1Z23J00018997,XCFD220,10282,9056457A134,ｼﾑ,100,（株）ムロコーポレーション,< NULL >,回収済,2023/9/8,...,2023/10/6 11:33,< NULL >,0.11,< NULL >,1.01,26.98,< NULL >,28.23,2023/10/6 11:33,< NULL >
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1Z23J00094385,XCGT760,40428,35300ECB010,ﾃﾞﾝﾄﾞｳｵｲﾙﾎﾟﾝﾌﾟ,12,アイシン精機（株）,半田工場,回収済,2023/9/29,...,2023/10/4 12:18,< NULL >,0.67,1.61,4.11,5.82,< NULL >,6.25,2023/10/4 12:18,< NULL >
296,1Z23J00091336,XCGR590,10415,9030119A010,ﾘﾝｸﾞO,1000,ＮＯＫ（株）,< NULL >,回収済,2023/9/29,...,2023/10/4 10:42,< NULL >,0.15,2.27,2.47,5.71,< NULL >,6.97,2023/10/4 10:42,< NULL >
297,1Z23J00065577,XCGL810,30468,G117362010,"ｾﾝｻｰ,ﾓｰﾀｰﾚﾎﾞﾘｭｰｼｮﾝ",18,多摩川精機販売（株）,< NULL >,回収済,2023/9/29,...,2023/10/4 10:42,< NULL >,0.11,3.31,3.51,7.73,< NULL >,8.01,2023/10/4 10:42,< NULL >
298,1Z23J00065411,XCGL200,30143,G1250ECE010,"ﾊﾟｲﾌﾟASSY,ﾓｰﾀｰｸｰﾘﾝｸﾞ",60,（株）バルカー,豊田営業所,回収済,2023/9/28,...,2023/10/4 10:42,< NULL >,0.09,3.04,3.15,8.45,< NULL >,8.85,2023/10/4 10:42,< NULL >


In [160]:
#確認用（実行は不要）
pre_zaikodata.isnull().sum()

かんばんシリアル                  0
伝票番号                      0
拠点所番地                     0
品番                        0
品名                        0
                       ... 
回収日時（HHMM）                0
不等ピッチ                  4368
納入予定時間                 4368
納入予定時間（HHMM）           4368
収時間と納入予定時間の差分（HHMM）    4368
Length: 69, dtype: int64

In [136]:
pre_zaikodata.head(300)

,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,...,社内LT（検収〜回収LT）/設計値_社内LT（検収〜回収LT）,検収日時（HHMM）,検収日時（HH:MM）,順立装置入庫日時（HHMM）,順立装置出庫日時（HHMM）,回収日時（HHMM）,不等ピッチ,納入予定時間,納入予定時間（HHMM）,収時間と納入予定時間の差分（HHMM）
0,1Z23J00061273,XCGG850,10429,35482TFA010,ｸﾗﾝﾌﾟｾﾝｻ-,1000,サトープレス工業（株）,本社工場,回収済,2023/9/26,...,3.128880,1017,10:17,1404,1104,-1,0.00,09:20:00,920.0,97.0
1,1Z23J00090101,XCGP260,10429,35482TFA010,ｸﾗﾝﾌﾟｾﾝｻ-,1000,サトープレス工業（株）,本社工場,順立装置内,2023/9/29,...,NaN,1750,17:50,1211,-1,-1,0.00,09:20:00,920.0,830.0
2,1Z23J00091003,XCGR590,10416,9030106A018,ﾘﾝｸﾞO,2000,ＮＯＫ（株）,< NULL >,順立装置内,2023/9/29,...,NaN,1754,17:54,1211,-1,-1,0.03,09:10:00,910.0,844.0
3,1Z23J00090128,XCGN430,10149,9056450A003,ｼﾑ,100,（株）ムロコーポレーション,< NULL >,回収済,2023/9/27,...,NaN,253,2:53,933,409,-1,0.00,02:00:00,200.0,53.0
4,1Z23J00018997,XCFD220,10282,9056457A134,ｼﾑ,100,（株）ムロコーポレーション,< NULL >,回収済,2023/9/8,...,NaN,-1,None,611,537,-1,0.00,02:00:00,200.0,-201.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1Z23J00094385,XCGT760,40428,35300ECB010,ﾃﾞﾝﾄﾞｳｵｲﾙﾎﾟﾝﾌﾟ,12,アイシン精機（株）,半田工場,回収済,2023/9/29,...,4.881321,2047,20:47,851,149,-1,0.14,21:00:00,2100.0,-53.0
296,1Z23J00091336,XCGR590,10415,9030119A010,ﾘﾝｸﾞO,1000,ＮＯＫ（株）,< NULL >,回収済,2023/9/29,...,2.963903,1754,17:54,2243,426,-1,0.03,09:10:00,910.0,844.0
297,1Z23J00065577,XCGL810,30468,G117362010,"ｾﾝｻｰ,ﾓｰﾀｰﾚﾎﾞﾘｭｰｼｮﾝ",18,多摩川精機販売（株）,< NULL >,回収済,2023/9/29,...,5.400554,1750,17:50,2246,353,-1,0.00,14:10:00,1410.0,340.0
298,1Z23J00065411,XCGL200,30143,G1250ECE010,"ﾊﾟｲﾌﾟASSY,ﾓｰﾀｰｸｰﾘﾝｸﾞ",60,（株）バルカー,豊田営業所,回収済,2023/9/28,...,5.805996,1515,15:15,1753,101,-1,0.24,14:30:00,1430.0,85.0


In [124]:
###LT

long = pre_zaikodata.iloc[:,1]
count = 0
for i in range(len(long)):
    if i % 5000==0:
            print(i)
    if ((pre_zaikodata.loc[i, '発注〜印刷LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜検収LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜回収LT'])!='< NULL >'):
    #if (not pd.isnull(pre_zaikodata.loc[i, '発注印刷LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注検収LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置入庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置出庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注回収LT'])):
        LT2 = float(pre_zaikodata.loc[i, '発注〜検収LT']) - float(pre_zaikodata.loc[i, '発注〜印刷LT'])
        LT3 = float(pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])-float(pre_zaikodata.loc[i, '発注〜検収LT'])
        LT4 = float(pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])-float(pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])
        LT5 = float(pre_zaikodata.loc[i, '発注〜回収LT'])-float(pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])
        LT6 = float(pre_zaikodata.loc[i, '発注〜回収LT'])-float(pre_zaikodata.loc[i, '発注〜検収LT'])
        pre_zaikodata.loc[i, '印刷検収LT'] = LT2
        pre_zaikodata.loc[i, '検収入庫LT'] = LT3
        pre_zaikodata.loc[i, '入庫出庫LT'] = LT4
        pre_zaikodata.loc[i, '出庫回収LT'] = LT5
        pre_zaikodata.loc[i, '社内LT（検収〜回収LT）'] = LT6
        count = count + 1
print(count)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
45256


In [125]:
###基準在庫枚数加味する
#文字型を整数型にする
#当月必要数？

#稼働日入力
#kadoubi=22

pre_zaikodata['回収日時'] = pd.to_datetime(pre_zaikodata['回収日時'], errors='coerce')

count = 0
tehaidata = pd.read_csv('..//data//設計値変更後_202309_手配数_1Y_2次.csv',encoding='cp932')
#tehaidata = pd.read_csv('sample3.csv',encoding='cp932')
long2 = tehaidata.iloc[:,1]

for i in range(len(long)):
    if ((pre_zaikodata.loc[i, '発注〜印刷LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜検収LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜回収LT'])!='< NULL >'):
    #if (not pd.isnull(pre_zaikodata.loc[i, '発注印刷LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注検収LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置入庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置出庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注回収LT'])):
        hinban = pre_zaikodata.loc[i,'品番'] 
        hinban2 = hinban.replace("-", "") 
        hinban3 = hinban2.replace(" ", "") 
        #print(i)
        if i % 5000==0:
                print(i)
        for j in range(len(long2)):
            #print(j)
            tehaihinban = tehaidata.loc[j,'品番'].replace('-', '')
            tehaihinban2 = tehaihinban.replace(' ', '')
            if hinban3 == tehaihinban2:
                day = pre_zaikodata.loc[i,'回収日時'].day
                index = tehaidata.columns.get_loc("翌々々月稼働日数") + day
                #count = pre_zaikodata.groupby('品番')['かんばんシリアル'].count()
                #pre_zaikodata.loc[i,'実かんばん数']　= 
                pre_zaikodata.loc[i,'日量数'] = int(str(tehaidata.iloc[j,index]).replace(',', ''))
                pre_zaikodata.loc[i,'日量数（箱数）'] = pre_zaikodata.loc[i,'日量数'] /pre_zaikodata.loc[i, '収容数']
                if int(str(tehaidata.loc[j,'当月必要数']).replace(',', '')) != 0:
                    avenichiryo = int(str(tehaidata.loc[j,'当月必要数']).replace(',', ''))/int(tehaidata.loc[j,'当月稼働日数'])
                    pre_zaikodata.loc[i,'平均日量数'] = avenichiryo
                    pre_zaikodata.loc[i,'平均日量数（箱数）'] = avenichiryo//pre_zaikodata.loc[i, '収容数']
                    pre_zaikodata.loc[i,'基準在庫枚数（日数）'] = (int(tehaidata.loc[j,'基準在庫枚数'])*pre_zaikodata.loc[i, '収容数'])/avenichiryo
                else:
                    pre_zaikodata.loc[i,'平均日量数'] = 0
                    pre_zaikodata.loc[i,'平均日量数（箱数）'] = 0
                    pre_zaikodata.loc[i,'基準在庫枚数（日数）'] = 0
print(count)

0
5000
10000
20000
25000
30000
35000
40000
45000
50000
0


In [126]:
count = 0
tehaidata = pd.read_csv('..//data//設計値変更後_XR10_手配運用情報2300911_1Y（回収引当適用）.csv',encoding='cp932')
#tehaidata = pd.read_csv('sample2.csv',encoding='cp932')
long2 = tehaidata.iloc[:,1]

for i in range(len(long)):
    if (not pd.isnull(pre_zaikodata.loc[i, '発注〜印刷LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注〜検収LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注〜順立装置入庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注〜順立装置出庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注〜回収LT'])):
        hinban = pre_zaikodata.loc[i,'品番'] 
        hinban2 = hinban.replace("-", "") 
        hinban3 = hinban2.replace(" ", "") 
        if i % 5000==0:
            print(i,hinban2)
        for j in range(len(long2)):
            tehaihinban = tehaidata.loc[j,'品番'].replace('-', '')
            tehaihinban2 = tehaihinban.replace(' ', '')
            #print(tehaihinban)
            if hinban3 == tehaihinban2:
                pre_zaikodata.loc[i, 'A'] = tehaidata.loc[j,'納入ｻｲｸﾙ間隔']
                pre_zaikodata.loc[i, 'B'] = tehaidata.loc[j,'納入ｻｲｸﾙ回数']
                pre_zaikodata.loc[i, 'C'] = tehaidata.loc[j,'納入ｻｲｸﾙ情報']
                pre_zaikodata.loc[i, '便Ave'] = pre_zaikodata.loc[i,'日量数（箱数）']/pre_zaikodata.loc[i, 'B']
                pre_zaikodata.loc[i, '整備室'] = tehaidata.loc[j,'整備室']
                pre_zaikodata.loc[i, '箱種類'] = tehaidata.loc[j,'登録箱種']
                pre_zaikodata.loc[i, '箱サイズ'] = tehaidata.loc[j,'容積(m3)']
                pre_zaikodata.loc[i, '箱重量'] = tehaidata.loc[j,'総重量(Kg)']
                #pre_zaikodata.loc[i, '不当ピッチ'] = tehaidata.loc[j,'不当ピッチ']
                #pre_zaikodata.loc[i, '通常品'] = tehaidata.loc[j,'通常品']
                #pre_zaikodata.loc[i, '選択品'] = tehaidata.loc[j,'選択品']
                pre_zaikodata.loc[i, '基準在庫日数'] = tehaidata.loc[j,'基準在庫日数']
                pre_zaikodata.loc[i, '基準在庫枚数'] = tehaidata.loc[j,'基準在庫枚数']
                pre_zaikodata.loc[i, '設計かんばん数'] = pre_zaikodata.loc[i,'日量数（箱数）'] + pre_zaikodata.loc[i, '基準在庫枚数']
                pre_zaikodata.loc[i, '手配区分'] = tehaidata.loc[j,'手配区分']
                MAX= tehaidata.loc[j,'基準在庫日数'] + float(pre_zaikodata.loc[i,'基準在庫枚数（日数）'])
                pre_zaikodata.loc[i,'基準在庫日数（基準在庫枚数込み）'] = MAX
                zisseki = pre_zaikodata.loc[i, '社内LT（検収〜回収LT）'] 
                if  (zisseki > MAX*0)and(zisseki < 0.25*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常？：設計値の0〜0.25倍'
                elif  (zisseki > MAX*0.25)and(zisseki < 0.5*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常？：設計値の0.25〜0.5倍'
                elif  (zisseki > MAX*0.5)and(zisseki < 0.75*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常よりの異常？：設計値の0.5〜0.75倍'
                elif  (zisseki > MAX*0.75)and(zisseki < MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常：設計値の0.75〜1倍'
                elif (zisseki >= MAX)and(zisseki < MAX*1.25):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常：設計値の1〜1.25倍'
                elif (zisseki >= MAX*1.25)and(zisseki < 1.5*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常よりの異常：設計値の1.25〜1.5倍'
                elif (zisseki >= MAX*1.5)and(zisseki < 2*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常：設計値の1.5〜2倍'
                elif (zisseki >= MAX*2):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常：設計値の2倍以上'
        #print(LT2,LT3,LT4,LT5)
        count = count + 1
print(count)

0 35482TFA010    
5000 9036652A004    
10000 41310ECB010    
15000 9036324A005    
20000 33490ECE010    
25000 41310ECE010    
30000 9036659A010    
35000 9036659A010    
40000 9010508A014    
45000 33490ECE010    
50000 35882ECB010    
51503


In [127]:
count = 0
sedata = pd.read_csv('..//data//設計値変更後_各工程の設計値.csv',encoding='cp932')
long2 = sedata.iloc[:,1]

for i in range(len(long)):
    if (not pd.isnull(pre_zaikodata.loc[i, '発注〜印刷LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注〜検収LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注〜順立装置入庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注〜順立装置出庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注〜回収LT'])):
        hinban = pre_zaikodata.loc[i,'品番']
        hinban2 = hinban.replace("-", "") 
        hinban3 = hinban2.replace(" ", "") 
        if i % 5000==0:
            print(i,hinban2)
        for j in range(len(long2)):
            sehinban = sedata.loc[j,'品番'].replace('-', '')
            sehinban2 = sehinban.replace(' ', '')
            #print(tehaihinban)
            if hinban3 == sehinban2:
                pre_zaikodata.loc[i, '設計値_検収入庫LT'] = sedata.loc[j,'検収入庫']
                pre_zaikodata.loc[i, '設計値_入庫出庫LT'] = sedata.loc[j,'入庫出庫']
                pre_zaikodata.loc[i, '設計値_出庫回収LT'] = sedata.loc[j,'出庫組立'] + sedata.loc[j,'引当回収']
                pre_zaikodata.loc[i, '設計値_出庫回収LT（基準在庫枚数込み）'] = sedata.loc[j,'出庫組立'] + sedata.loc[j,'引当回収'] + pre_zaikodata.loc[i,'基準在庫枚数（日数）']
                pre_zaikodata.loc[i, '検収入庫LT/設計値_検収入庫LT'] = pre_zaikodata.loc[i, '検収入庫LT']/sedata.loc[j,'検収入庫']
                pre_zaikodata.loc[i, '入庫出庫LT/設計値_入庫出庫LT'] = pre_zaikodata.loc[i, '入庫出庫LT']/sedata.loc[j,'入庫出庫']
                pre_zaikodata.loc[i, '出庫回収LT/設計値_出庫回収LT'] = pre_zaikodata.loc[i, '出庫回収LT']/(sedata.loc[j,'出庫組立'] + sedata.loc[j,'引当回収'])
                pre_zaikodata.loc[i, '出庫回収LT（基準在庫枚数込み）/設計値_出庫回収LT'] = pre_zaikodata.loc[i, '出庫回収LT']/(sedata.loc[j,'出庫組立'] + sedata.loc[j,'引当回収'] + pre_zaikodata.loc[i,'基準在庫枚数（日数）'])
                pre_zaikodata.loc[i, '社内LT（検収〜回収LT）/設計値_社内LT（検収〜回収LT）'] = pre_zaikodata.loc[i, '社内LT（検収〜回収LT）']/pre_zaikodata.loc[i,'基準在庫日数（基準在庫枚数込み）']

0 35482TFA010    
5000 9036652A004    
10000 41310ECB010    
15000 9036324A005    
20000 33490ECE010    
25000 41310ECE010    
30000 9036659A010    
35000 9036659A010    
40000 9010508A014    
45000 33490ECE010    
50000 35882ECB010    


In [128]:
# 日付型の時間と分（HH-MM）を数値型（HHMM）に変換する関数
def extract_hour_minute_numeric(time_str):
    try:
        time_parts = time_str.split()[1].split(':')
        return float(time_parts[0] + time_parts[1])
    except:
        return None

# 検収日時列から時間と分を取り出し、新しい列に追加
pre_zaikodata['検収日時（HHMM）'] = pre_zaikodata['検収日時'].apply(extract_hour_minute_numeric)
# NaNの値を-1に置き換える
pre_zaikodata['検収日時（HHMM）'].fillna(-1, inplace=True)
# 列を整数型に変換
pre_zaikodata['検収日時（HHMM）'] = pre_zaikodata['検収日時（HHMM）'].astype(int)

def extract_HHMM_from_datetime(datetime_str):
    if pd.isna(datetime_str) or datetime_str == "< NULL >":
        return None
    return datetime_str.split(" ")[-1]

# Apply the extraction to the "検収日時" column
pre_zaikodata["検収日時（HH:MM）"] = pre_zaikodata["検収日時"].apply(extract_HHMM_from_datetime)


# 順立装置入庫列から時間と分を取り出し、新しい列に追加
pre_zaikodata['順立装置入庫日時（HHMM）'] = pre_zaikodata['順立装置入庫日時'].apply(extract_hour_minute_numeric)
# NaNの値を-1に置き換える
pre_zaikodata['順立装置入庫日時（HHMM）'].fillna(-1, inplace=True)
# 列を整数型に変換
pre_zaikodata['順立装置入庫日時（HHMM）'] = pre_zaikodata['順立装置入庫日時（HHMM）'].astype(int)

# 順立装置出庫列から時間と分を取り出し、新しい列に追加
pre_zaikodata['順立装置出庫日時（HHMM）'] = pre_zaikodata['順立装置出庫日時'].apply(extract_hour_minute_numeric)
# NaNの値を-1に置き換える
pre_zaikodata['順立装置出庫日時（HHMM）'].fillna(-1, inplace=True)
# 列を整数型に変換
pre_zaikodata['順立装置出庫日時（HHMM）'] = pre_zaikodata['順立装置出庫日時（HHMM）'].astype(int)

# 回収日時列から時間と分を取り出し、新しい列に追加
pre_zaikodata['回収日時（HHMM）'] = pre_zaikodata['回収日時'].apply(extract_hour_minute_numeric)
# NaNの値を-1に置き換える
pre_zaikodata['回収日時（HHMM）'].fillna(-1, inplace=True)
# 列を整数型に変換
pre_zaikodata['回収日時（HHMM）'] = pre_zaikodata['回収日時（HHMM）'].astype(int)

# 結果の確認
pre_zaikodata[['検収日時','検収日時（HH:MM）', '検収日時（HHMM）','順立装置入庫日時','順立装置入庫日時（HHMM）','順立装置出庫日時','順立装置出庫日時（HHMM）','回収日時','回収日時（HHMM）']].head(30)

,検収日時,検収日時（HH:MM）,検収日時（HHMM）,順立装置入庫日時,順立装置入庫日時（HHMM）,順立装置出庫日時,順立装置出庫日時（HHMM）,回収日時,回収日時（HHMM）
0,2023/9/26 10:17,10:17,1017,2023/9/26 14:04,1404,2023/10/2 11:04,1104,2023-10-06 12:18:00,-1
1,2023/9/29 17:50,17:50,1750,2023/10/6 12:11,1211,< NULL >,-1,NaT,-1
2,2023/9/29 17:54,17:54,1754,2023/10/6 12:11,1211,< NULL >,-1,NaT,-1
3,2023/9/28 2:53,2:53,253,2023/9/28 9:33,933,2023/10/6 4:09,409,2023-10-06 11:33:00,-1
4,< NULL >,None,-1,2023/9/9 6:11,611,2023/10/5 5:37,537,2023-10-06 11:33:00,-1
5,2023/9/28 16:33,16:33,1633,2023/9/29 7:28,728,2023/10/5 14:13,1413,2023-10-06 11:33:00,-1
6,2023/9/28 2:53,2:53,253,2023/9/28 9:38,938,2023/10/6 11:09,1109,NaT,-1
7,2023/9/20 2:42,2:42,242,2023/9/20 6:33,633,2023/10/6 11:09,1109,NaT,-1
8,2023/9/28 23:25,23:25,2325,2023/9/29 5:51,551,2023/10/5 23:58,2358,2023-10-06 10:49:00,-1
9,2023/9/27 18:30,18:30,1830,2023/9/28 6:36,636,2023/10/6 10:13,1013,NaT,-1


In [130]:
from datetime import datetime

# Load the new CSV file into a DataFrame with 'Shift_JIS' encoding
df_pitch = pd.read_excel('../data/仕入れ先納入便ダイヤ.xlsx', encoding='Shift_JIS')
# Display the first few rows of the DataFrame
df_pitch.head()

long2 = df_pitch.loc[:,"仕入先名"]

# "1便"の列から時間と分を取り出して整数型のHHMMに変換する関数（引数として文字列を受け取るバージョン）
def convert_string_to_HHMM(time_str):
    if pd.isna(time_str):
        return None
    # 文字列をdatetime.timeオブジェクトに変換
    time_obj = datetime.strptime(time_str, '%H:%M:%S').time()
    return time_obj.hour * 100 + time_obj.minute

# Convert "検収日時（HH:MM）" to datetime.time object
def string_to_time(time_str):
    if pd.isna(time_str):
        return None
    dt_obj = datetime.strptime(time_str, '%H:%M')
    return dt_obj.time()

def convert_to_short_time_format(time_str):
    # Convert the string to a datetime object
    dt_obj = datetime.strptime(time_str, '%H:%M:%S')
    # Convert the datetime object back to string in the desired format
    return dt_obj.strftime('%-H:%M')

#pre_zaikodata["検収日時（HH：MM）"] = pre_zaikodata["検収日時（HH:MM）"].apply(string_to_time)

for i in range(len(long)):
    if i % 5000==0:
            print(i)
    for j in range(len(long2)):
        #（株）ＢｌｕＥ　Ｎｅｘｕｓは入庫されていない、存在しない6便があるけど
        if ((pre_zaikodata.loc[i, '仕入先名'] == df_pitch.loc[j,'仕入先名']) and (df_pitch.loc[j, '観点'] == "着時刻") and pre_zaikodata.loc[i, '仕入先名'] != "（株）ＢｌｕＥ　Ｎｅｘｕｓ"):
            #print(df_design.loc[i, '仕入先名/工場名'],df_pitch.loc[j,'仕入先名']
            pre_zaikodata.loc[i,"不等ピッチ"] = df_pitch.loc[j,"不等ピッチ係数日"]
            track = int(pre_zaikodata.loc[i, '納入便'])
            index = df_pitch.columns.get_loc("観点") + track
            pre_zaikodata.loc[i,"納入予定時間"] = df_pitch.iloc[j,index]
            pre_zaikodata.loc[i,"納入予定時間（HHMM）"] = convert_string_to_HHMM(df_pitch.iloc[j,index].strftime('%H:%M:%S'))
            #print(i,j,pre_zaikodata.loc[i,"納入予定時間"],pre_zaikodata.loc[i,"検収日時（HH:MM）"],pre_zaikodata.loc[i, '仕入先名'])
            #pre_zaikodata.loc[i,"検収時間と納入予定時間の差分"] = (pre_zaikodata.loc[i,"検収日時（HH:MM）"] - pre_zaikodata.loc[i,"納入予定時間"])/60
            #print(i,j,pre_zaikodata.loc[i,"納入予定時間"],pre_zaikodata.loc[i,"検収日時（HH:MM）"],pre_zaikodata.loc[i, '仕入先名'])


0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000


In [135]:
# Calculate the time difference again
pre_zaikodata['検収時間と納入予定時間の差分（HHMM）'] = pre_zaikodata['検収日時（HHMM）'] - pre_zaikodata['納入予定時間（HHMM）']

In [138]:
with open('..//data//設計値変更後_LTデータ9月.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    pre_zaikodata.to_csv(f)

In [158]:
#指定日時を取り出す

pre_zaikodata['回収日時'] = pd.to_datetime(pre_zaikodata['回収日時'], errors='coerce')
pre_zaikodata['回収日時']    = pd.to_datetime(pre_zaikodata['回収日時'], errors='coerce')
pre_zaikodata['納入日'] = pd.to_datetime(pre_zaikodata['納入日'], errors='coerce')
pre_zaikodata['納入日']    = pd.to_datetime(pre_zaikodata['納入日'], errors='coerce')

#月から土曜の9時まで
pre_zaikodata2 = pre_zaikodata[((pre_zaikodata['納入日'] >= '2023-09-04') & 
                              (pre_zaikodata['回収日時'] <= '2023-09-09-10')) | ((pre_zaikodata['納入日'] >= '2023-09-11') & 
                              (pre_zaikodata['回収日時'] <= '2023-09-16-10')) | ((pre_zaikodata['納入日'] >= '2023-09-18') & 
                              (pre_zaikodata['回収日時'] <= '2023-09-23-10')) | ((pre_zaikodata['納入日'] >= '2023-09-25') & 
                              (pre_zaikodata['回収日時'] <= '2023-09-30-10'))]

#田中さんから頂いた3月データはシムが含まれている。1Yでないものもある
print("設計値変更後：","元データのかんばん数",len(pre_zaikodata),"抽出したデータのかんばん数",len(pre_zaikodata2),"元データの品番数",len(pre_zaikodata.loc[:,'品番'].unique()),"抽出したデータの品番数",len(pre_zaikodata2.loc[:,'品番'].unique()))

設計値変更後： 元データのかんばん数 51503 抽出したデータのかんばん数 37304 元データの品番数 202 抽出したデータの品番数 157


In [162]:
with open('..//data//設計値変更後_LTデータ9月（土日削除）.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    pre_zaikodata3.to_csv(f)

In [54]:
pre_zaikodata["回収日時"].head(300)

0     2023-10-06 12:18:00
1                     NaT
2                     NaT
3     2023-10-06 11:33:00
4     2023-10-06 11:33:00
              ...        
295   2023-10-04 12:18:00
296   2023-10-04 10:42:00
297   2023-10-04 10:42:00
298   2023-10-04 10:42:00
299   2023-10-04 10:42:00
Name: 回収日時, Length: 300, dtype: datetime64[ns]

In [9]:
7/3

2.3333333333333335

In [74]:
df_pitch.head()

,Unnamed: 0,仕入先\nコード,工区,仕入先名,発送場所名,受入,納入先,観点,1便,2便,...,23便,24便,最長便ピッチ時間\n【h:mm】,"最長便ピッチを\n""分""へ変換\n【分】",納入回数\n【回】,等ピッチ時間\n【分】,不等ピッチ時間\n【分】,1日1回納入省く\n【分】,不等ピッチ係数日,Unnamed: 39
0,112.0,1.0,12.0,アイシン精機（株）,半田工場,1Y,カリツー西尾東経由,着時刻,10:30:00,14:40:00,...,NaN,NaN,07:20:00,440.0,4.0,287.5,152.5,152.5,0.14,NaN
1,NaN,1.0,12.0,アイシン精機（株）,半田工場,1Y,カリツー西尾東経由,便間,03:05:00,04:50:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2401.0,24.0,1.0,（株）青山製作所,NaN,1Y,カリツー西尾東経由,着時刻,07:00:00,11:20:00,...,NaN,NaN,04:35:00,275.0,6.0,191.7,83.3,83.3,0.08,NaN
3,NaN,24.0,1.0,（株）青山製作所,NaN,1Y,カリツー西尾東経由,便間,04:10:00,02:30:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3001.0,30.0,1.0,（株）浅賀井製作所,安城工場,1Y,カリツー西尾東経由,着時刻,09:50:00,15:10:00,...,NaN,NaN,07:10:00,430.0,4.0,287.5,142.5,142.5,0.13,NaN


In [161]:
pre_zaikodata3 =pre_zaikodata2.drop(["伝票番号","拠点所番地","ステータス","仕入先工場名","発注日時"],axis=1)